<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 36px">Notebook de exemplo para como corretamente fazer o ETL dos dados</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 22px; padding: 5px; border-radius: 5px;">
  <li>Apresenta apenas como organizar os dados com um exemplo em JSON.</li>
  <li>Quais devem ser os atributos para envio no Postgre DB.</li>
</ul>

In [6]:
import pandas as pd
import numpy as np
import os, json
import itertools

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Exemplo Calendar API</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Como coletar os dados das interações dos participantes em uma meeting</li>
  <li>Como criar as interações entre os participantes de cada meeting dos dados extraídos</li>
</ul>
<br>
Pontos de atenção
<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>O sender e receiver devem manter essa nomeclatura mesmo quando o sender é um participante da reunião. Precisamos criar as interações entre cada colaborador</li>
  <li>Esse código é apenas um exemplo, mas os resultados devem estar exatamente iguais para garantirmos a replicação correta dos dados</li>
</ul>

In [7]:
# Importando os dados de exemplo do Calendar API
# Obviamente esses dados serão extraídos diretamente da chamada da Calendar API, aqui é apenas um exemplo

with open('calendar_data_example.json', 'r') as file:
    # Coletando os dados
    # No seu caso aqui ao inves de json.load, teríamos o .get da API do Calendar API.
    data = json.load(file)
    result_calendar = []
    # Aqui acessamos os dados de calendário de cada usuário que extraímos.
    # No seu caso, você vai estar coletando o nome de cada usuário e extraindo seu calendário
    for data_item in data['calendars']:
        # Aqui nos coletamos os items/meetings dentro da agenda do nosso colaborador, cada meeting corresponde ao período que você utilizou quando requisitou os dados no .get()
        for meeting in data_item['items']:
            # Aqui nos coletamos todos os participantes da agenda e adicionamos quem enviou a agenda, precisamos utilizar o set() para eliminar duplicatas
            users_agenda = set([person['email'] for person in meeting['attendees']] + [data_item['summary']])
            # Aqui nos realizamos a combinação de cada participante 2 a 2, ou seja podemos ter: anfitrião > participante, como participante > participante
            # Isso é muito importante para garantir os dados de interação de cada pessoa da reunião.
            combinations = list(itertools.combinations(users_agenda, 2))
            # Depois disso, realizamos o processo de salvar o resultado.
            result_calendar.extend([
                {
                 'id':meeting['id'],
                 'sender':dupla[0],
                 'receiver':dupla[1],
                 'data': pd.Timestamp(meeting['start']['dateTime']).date(),
                 'de_onde':'calendar_api'
                }
                for dupla in combinations
            ])

# Resultado da extração de 2 reuniões.
result_calendar = pd.DataFrame(result_calendar) ; result_calendar

,id,sender,receiver,data,de_onde
0,event1,user2@company.com,user3@company.com,2025-03-05,calendar_api
1,event1,user2@company.com,user1@company.com,2025-03-05,calendar_api
2,event1,user2@company.com,user4@company.com,2025-03-05,calendar_api
3,event1,user2@company.com,user5@company.com,2025-03-05,calendar_api
4,event1,user3@company.com,user1@company.com,2025-03-05,calendar_api
5,event1,user3@company.com,user4@company.com,2025-03-05,calendar_api
6,event1,user3@company.com,user5@company.com,2025-03-05,calendar_api
7,event1,user1@company.com,user4@company.com,2025-03-05,calendar_api
8,event1,user1@company.com,user5@company.com,2025-03-05,calendar_api
9,event1,user4@company.com,user5@company.com,2025-03-05,calendar_api


<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, temos casos de ID iguais com senders e receivers iguais, precisamos resolver esse problema!</li>
  <li>Isso acontece pois extraímos os dados de dois calendários de users diferentes, trazendo a mesma reunião e por isso precisamos eliminar essa duplicação para isso criamos um hash_id que mapeia a relação</li>
</ul>

In [9]:
# Utilize dessa função para criar o hashmap dos colaboradores da organização
# Queremos mapear a relação entre 2 colaboradores independente da ordem das colunas
import hashlib
def generate_hash(sender, receiver):
    interaction = f"{min(sender, receiver)}-{max(sender, receiver)}"
    return hashlib.sha256(interaction.encode()).hexdigest()

result_calendar['hash_id'] = result_calendar.apply(lambda row: generate_hash(row['sender'], row['receiver']), axis=1)

In [10]:
result_calendar[result_calendar['id'] == 'event2']['hash_id'].value_counts()

1cbfc5abcc325a852e83c053c2e07fbe51e72a0f714fbe0c636e143108bdc424    2
bccd40a33ca2d17f86a999dc6fcb17475e351a2abe1cd4cb9e8cd165f616aa61    2
2c6c91b40e8067ee83c433e700fb12f794843652a2d4a0d2c58551d702750d9f    2
42b964b342866bbbfc78daa34724128ebbe185a7e7badb4a36bdebbcfcfefedf    2
6b70d49e912a48fb6d8f1b88bc56e08498d803310023b5930126adaf1422b62e    2
f3392d571efc59c3d57112fe9b115f3deaca69e1fb22c2e5132553c8036c7846    2
3cddfeb6f5c60f7179a9184b86ecfd0600594470cf81ddea131f38acabfa9728    2
61da940bf15029fa40308daf5c1561bc85fd5a5d14f1b9118a1b9ffbe8370bb7    2
683e0c1d2b6e261d07b310ce024131bde9c96927afb41f94fdeb993d81d41454    2
a2967b493347c2af442baf8455c1cf1617f1798b43c02ad467d79b6e8ef26f6c    2
Name: hash_id, dtype: int64

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, como realmente para um ID temos 2 hash_ids iguais, precisamos agrupar e eliminar duplicatas</li>
</ul>

In [11]:
result_calendar.groupby(['id','hash_id']).agg(
    sender=('sender','max'),
    receiver=('receiver','max'),
    data=('data','max'),
    de_onde=('de_onde','max')
).reset_index()

,id,hash_id,sender,receiver,data,de_onde
0,event1,2c6c91b40e8067ee83c433e700fb12f794843652a2d4a0...,user2@company.com,user4@company.com,2025-03-05,calendar_api
1,event1,42b964b342866bbbfc78daa34724128ebbe185a7e7badb...,user2@company.com,user5@company.com,2025-03-05,calendar_api
2,event1,61da940bf15029fa40308daf5c1561bc85fd5a5d14f1b9...,user3@company.com,user4@company.com,2025-03-05,calendar_api
3,event1,683e0c1d2b6e261d07b310ce024131bde9c96927afb41f...,user3@company.com,user5@company.com,2025-03-05,calendar_api
4,event1,73c44ef17a2088e45e038792c81127c5a39e71dbe116f7...,user1@company.com,user5@company.com,2025-03-05,calendar_api
5,event1,78bd6cde6e1080e1a0aec4d70099aecdef5cecb0213596...,user1@company.com,user4@company.com,2025-03-05,calendar_api
6,event1,a2967b493347c2af442baf8455c1cf1617f1798b43c02a...,user4@company.com,user5@company.com,2025-03-05,calendar_api
7,event1,aedf17d1a4886adc4ac119c3c24b60bf4a094860cfa92d...,user2@company.com,user1@company.com,2025-03-05,calendar_api
8,event1,bccd40a33ca2d17f86a999dc6fcb17475e351a2abe1cd4...,user2@company.com,user3@company.com,2025-03-05,calendar_api
9,event1,c70f788284647680a60dd030bec8f69fcd9438fc1b9bd3...,user3@company.com,user1@company.com,2025-03-05,calendar_api


### Critérios de aceite:

* É possível existir id duplicados.
* É possível existir hash_id duplicados.
* Não se pode possuir id e hash_id duplicados.
* É necessário realizar as combinações par a par de todos os participantes da reunião utilizando o itertools como foi demonstrado.